In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

In [ ]:
import csv

# Define the input and output file names
input_file_name = 'cleaned_file_hourly.csv'
output_file_name = 'cleaned_smooth_time.csv'

# Open the input CSV file and read its contents into a dictionary
with open(input_file_name, 'r') as input_file:
    reader = csv.DictReader(input_file)
    data = [row for row in reader]

# Initialize a dictionary to hold the output data
output_data = {}

# Iterate over each row in the input data and calculate the sums for each id and date
for row in data:
    id = row['id']
    date = row['date']
    value_step_goal = int(float(row['step_goal']))
    hour = int(float(row['hour']))
    value = int(float(row['steps']))
    gender = row['gender']
    age = row['age']
    bmi = row['bmi']
    if id not in output_data:
        output_data[id] = {}
    if date not in output_data[id]:
        output_data[id][date] = {'sum_steps_0_to_6': 0, 'sum_steps_6_to_12': 0, 'sum_steps_12_to_16': 0, 'gender': gender, 'age': age, 'bmi': bmi, 'step_goal':0, 'step_walked': 0}
    if hour <= 6:
        output_data[id][date]['sum_steps_0_to_6'] += value
    elif hour <= 12:
        output_data[id][date]['sum_steps_6_to_12'] += value
    elif hour <=16:
        output_data[id][date]['sum_steps_12_to_16'] += value
    else:
        pass
    output_data[id][date]['step_goal'] += value_step_goal
    output_data[id][date]['step_walked'] += value

# Write the output data to a new CSV file
with open(output_file_name, 'w', newline='') as output_file:
    fieldnames = ['id', 'date', 'gender', 'age', 'bmi', 'sum_steps_0_to_6', 'sum_steps_6_to_12', 'sum_steps_12_to_16', 'step_goal', 'step_walked', 'goal_reached']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()
    for id in output_data:
        for date in output_data[id]:
            if (output_data[id][date]['sum_steps_0_to_6'] != 0 or output_data[id][date]['sum_steps_6_to_12'] != 0 or output_data[id][date]['sum_steps_12_to_16'] != 0) and output_data[id][date]['step_goal'] != 0:
                row = {
                    'id': id,
                    'date': date,
                    'gender': output_data[id][date]['gender'],
                    'age': output_data[id][date]['age'],
                    'bmi': output_data[id][date]['bmi'],
                    'sum_steps_0_to_6': output_data[id][date]['sum_steps_0_to_6'],
                    'sum_steps_6_to_12': output_data[id][date]['sum_steps_6_to_12'],
                    'sum_steps_12_to_16': output_data[id][date]['sum_steps_12_to_16'],
                    'step_goal': output_data[id][date]['step_goal'],
                    'step_walked': output_data[id][date]['step_walked'],
                    'goal_reached': output_data[id][date]['step_walked'] >= output_data[id][date]['step_goal']
                }
                writer.writerow(row)
